In [12]:
from core.model import get_clip_classifier_from_pretrained
from torchsummary import summary

In [71]:
from templates.templates import Templates

template_name = 'fashion_templates'

if hasattr(Templates(), template_name):
    templates = getattr(Templates(), template_name)
else:
    raise Exception(f"Template {template_name} not found.")

In [72]:
# path for dataset.
import yaml
from pathlib import Path

with open(Path('config') / Path('data' + '.yaml'), "r") as _data:
    data = yaml.load(_data, Loader=yaml.FullLoader)
    _data.close()

class_dict = data['names']

In [73]:
class_dict

{0: 'dress', 1: 'jumpsuit', 2: 'pants', 3: 'skirt', 4: 'blouse'}

In [74]:
image_classifier = get_clip_classifier_from_pretrained(clip_path = 'ViT-B/16',  # X:\Users\user\.cache\clip\ViT-B_16.pt
                                                  class_dict = class_dict, 
                                                  templates = templates, 
                                                  freeze_encoder= True,
                                                  device = 'cuda')

Getting zeroshot weights.


100%|██████████| 5/5 [00:00<00:00, 29.88it/s]


In [75]:
from PIL import Image

In [76]:
image_features = image_classifier.val_preprocess(Image.open(r"D:\Datasets\TCC\pipe_debug\test\skirt\Abstract_Brushstroke_Print_Pencil_Skirt-img_00000022.png"))
image_classifier.eval()
image_features = image_features.unsqueeze(0).to('cuda')

In [77]:
logits = image_classifier(image_features)

In [78]:
class_dict

{0: 'dress', 1: 'jumpsuit', 2: 'pants', 3: 'skirt', 4: 'blouse'}

In [79]:
logits

tensor([[25.5135, 27.4516, 25.6006, 25.7649, 27.8653]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [80]:
import torch

In [81]:
m = torch.nn.Softmax(dim=-1)
m(logits)

tensor([[0.0480, 0.3335, 0.0524, 0.0617, 0.5044]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [82]:
class_dict[logits.argmax(dim=-1).item()]

'blouse'

----

inference

In [1]:
import torch

model = torch.load('zero_shot_models/vit_b_16_df1_16.pth')

In [2]:
from utils.val import val
from core.loader import batch_loader

val_data_loader = batch_loader(csv_file='/home/vinicius-cin/Datasets/DF1_16c_classification_filtered_v1/test/test.csv', 
                                        root_dir='/home/vinicius-cin/Datasets/DF1_16c_classification_filtered_v1', 
                                        transform=model.val_preprocess, 
                                        job_type = 'multiclass',
                                        batch_size=64, num_workers=1, 
                                        is_clip=True, shuffle=False, 
                                        generator=None, worker_init_fn=None)

In [3]:
# @ Validation metrics.
val_results = val(model, loader = val_data_loader, criterion = None, 
                    calc_loss = False, arch = {'job_type' : 'multiclass', 'out_features' : 16}, device='cuda', metrics_conf = {'top_k':3, 'num_samples': 10})

val_metrics, val_loss = val_results['metrics'], val_results['loss']

In [4]:
val_metrics.precision, torch.mean(val_metrics.precision)

(tensor([0.3400, 0.7850, 0.7200, 0.6500, 0.7100, 0.7200, 0.5600, 0.5550, 0.4150,
         0.8350, 0.4850, 0.8250, 0.7650, 0.6150, 0.0050, 0.7600],
        device='cuda:0'),
 tensor(0.6091, device='cuda:0'))